# Lab Assignment Six: Wide and Deep Networks



__*Austin Chen, Luke Hansen, Oscar Vallner*__

## 1. Preparation and Overview


For the 5th Lab Assignment, we will be using the same Federal Employment Dataset from Lab 4 and 5. Though we are using the same data, we have chosen to tackle a different classification problem. Instead of trying to predict a government employee's education level, we will be attempting to predict an employee's compensation within a certain range.

### 1.1 Business Understanding